In [1]:
import numpy as np
import matplotlib.pyplot as plt

### The Big Problem

In *B - Location optimisation with Gaussian Processes* I looked at how you can choose a good location while minimizing the number of samples that you take. The solution was to use a Gaussian process and maximize the expected improvement at each sample. Now we turn to a related problem. In the real world, robots are not limited by the number of samples, but rather by the size of their batteries. Taking samples and moving both incur costs. Given a fixed energy budget, how do you determine where to sample?

One paper (https://arxiv.org/pdf/1401.3462) looks at how to coordinate the trajectory of multiple agents exploring some function by maximizing the information gained. If each robot $i$ travels some path $P_i$, sampling at each location, we want to choose these paths to maximize the information obtained, subject to an energy budget:
$$
\max_{P_1,\dots,P_n} I(\text{observations along } \cup_i P_i)
\quad \text{subject to} \quad C(P_i) \le B \;\; \forall i,
$$
where $B$ is the available budget and $I(\cdot)$ measures the information gained from the samples.

### A brief detour into information theory

'Information'. What is it? Well, imagine I am looking out the window. You are sitting in the house. I notice two things. First, that it's sunny. Secondly, that a squirrel is running up the tree in the front yard. Both of these statements contain information. But notice that the second one takes many more words to say. That's because it is more specific. A squirrel running up a tree is one of many different things that could be happening in the front yard. The weather is only going to be one of a few things. You can think of information as the number of words or the amount of communication I need to tell you what the sample is from a distribution. The distribution of weather has a smaller range, so I need fewer words to describe it. 

Take the example of flipping a coin. Say I flip it 2 times and I need to tell you what the results were. How many words do I need? Well, it's exactly 2. 2 is the information content I have after observing the coin flipped twice. But now consider a skewed distribution. Say that I get heads 90% of the time. Then getting heads twice is very common and I should probably use fewer words to tell you that the usual thing has happened. Imagine I say "Usual" if I get heads twice, and otherwise I say "Unusual" followed by the results if I don't. How many words do I use now? Well, instead of 2 words every time, the expected number is $0.81 + 0.19 \times 3 = 1.38$, much lower! The reason is that with the *shared* knowledge of the distribution and its skew I have a lower amount of information to communicate to you. Information reflects the surprise upon seeing a particular result.

Now we want to formalize this. One property we would like is that information should add for independent events. That is, if I learn the outcome of two independent events, the total information gained should be the sum of the information gained from each one separately. For independent events, probabilities multiply. So we are looking for a function that converts products of probabilities into sums of numbers. The only function with this property (up to a scaling factor) is the logarithm.

This motivates defining the information content of an event with probability $p$ as
$$
\log \frac{1}{p} = -\log p.
$$

In simpler terms the number of words I need to describe something happening grows logarithmically with how unlikely it is. 

Next we can look at entropy, which is just the expected information for some set of discrete events $X$:
$$
H(X) = -\sum_{x\in X} p(x) \log p(x)
$$

And we get mutual information, which is the reduction in uncertainty about one variable after observing another:
$$
I(X;Y) = H(X) - H(X|Y)
$$

Here $X$ could be your test data and $Y$ your known data. In the location optimization context, $Y$ is where you choose to explore and $X$ is what remains unknown.

Consider the number of unique points you can specify with each bit of information. With 1 bit you can specify 2 points (0 and 1) with 2 you can specify 4. With 3 you can specify 8.

### Back to the problem

Now it's worth wondering why we are using information gained as the metric here rather than, say, the maximum value (which is what we actually care about). The reason is that mutual information is always going to increase with each sample taken and the returns (change in information) will be diminishing with each sample. This makes it easier to justify a greedy approach. 